In [1]:
import ee
from IPython.display import YouTubeVideo
import os
import geemap
import pandas as pd
from geemap.datasets import DATA
import matplotlib.pyplot as plt
from osgeo import gdal
from geemap import geojson_to_ee, ee_to_geojson
import json
import os
import requests
import folium
import ipywidgets as widgets
from ipywidgets import Text, HTML
from ipyleaflet import WidgetControl, MarkerCluster, LayersControl, Popup
from ipyleaflet import AwesomeIcon, GeoJSON, Map, Marker, LayerGroup, SearchControl, FullScreenControl, ZoomControl

In [2]:
ee.Initialize()
# ee.Authenticate()

In [3]:
pred = ee.Image('users/ashamba/Pred_Map')
collines = ee.Image('users/ashamba/map_collines')
states = ee.FeatureCollection("TIGER/2018/States")
benin_adm0 = ee.FeatureCollection("users/ashamba/BEN_adm0")
benin_adm1 = ee.FeatureCollection("users/ashamba/BEN_adm1")
benin_adm2 = ee.FeatureCollection("users/ashamba/BEN_adm2")

In [4]:
collectionFromImages = ee.ImageCollection.fromImages([pred, collines])

In [5]:
benin_adm0_json = ee_to_geojson(benin_adm0)
benin_adm1_json = ee_to_geojson(benin_adm1)
benin_adm2_json = ee_to_geojson(benin_adm2)

In [6]:
json_layer_ben0 = GeoJSON(data=benin_adm0_json, name='Benin Country JSON', hover_style={'fillColor': 'green' , 'fillOpacity': 0.5})
json_layer_ben1 = GeoJSON(data=benin_adm1_json, name='Benin Departments JSON', hover_style={'fillColor': 'green' , 'fillOpacity': 0.5})
json_layer_ben2 = GeoJSON(data=benin_adm2_json, name='Benin Communes JSON', hover_style={'fillColor': 'green' , 'fillOpacity': 0.5})

In [7]:
ben_map = geemap.Map(zoom=7, center=[9.0, 2.4],zoom_control=False)
ben_map.clear_controls()
benin_bounds = widgets.Dropdown(
    options=[('Select Boundary', 0), ('Benin Republic', 1), ('Benin Departments', 2), ('Benin Communes', 3)],
    layout={'width': 'max-content'},
    value=0,
    description='',
)

ben_map.add_control(FullScreenControl(position='topright'))
ben_map.add_control(ZoomControl(position='topright'))
ben_map.add_control(LayersControl(position='topright'))

html1 = HTML('''
    <h4>Map Information Terminal</h4>
    <h5>Move pointer to any region in Benin</h5>
''')
html1.layout.margin = '0px 20px 20px 20px'
# control1 = WidgetControl(widget=html1, position='bottomright')
# ben_map.add_control(control1)

# class Update_Features():
#     def __init__(self):
#         self.name = ""
#     def update_department(self, feature, **kwargs):
#         self.name = feature['properties']['NAME_1']
#         html1.value = '''
#             <h4>Admin Type: <b>{}</b></h4>
#             <h6>Department Name: <b>{}</b></h6>
#             <h6>Cashew Yield: <b>{}</b></h6>
#         '''.format(feature['properties']['ID_0'], feature['properties']['NAME_1'], "Alot can go here")
#     def update_communes(self, feature, **kwargs):
#         self.name = feature['properties']['NAME_1']
#         html1.value = '''
#             <h4>Admin Type: <b>{}</b></h4>
#             <h6>Department Name: <b>{}</b></h6>
#             <h6>Cashew Yield: <b>{}</b></h6>
#         '''.format(feature['properties']['ID_0'], feature['properties']['NAME_2'], "Alot can go here")
                   
# update_feat = Update_Features()
# json_layer_ben1.on_click(update_feat.update_department)
# json_layer_ben2.on_click(update_feat.update_communes)

In [8]:
dist_stats = collectionFromImages.reduce(reducer = ee.Reducer.sum()).eq(1).reduceRegions(
  collection = benin_adm2,
  reducer = ee.Reducer.sum(),
  scale = 1000
)
dist_stats = dist_stats.select(['NAME_0','NAME_1', 'NAME_2', 'sum'], ['Country', 'Districts', 'Communes','Cashew_Yield'], retainGeometry=True).getInfo()

In [9]:
dtstats_df = pd.DataFrame()
for dist in dist_stats['features']:
    df = pd.DataFrame([dist['properties']], columns=dist['properties'].keys())
    dtstats_df = pd.concat([dtstats_df, df], axis=0)

In [10]:
# Reorder columns in dataframe, sort by state name and reset index
dtstats_df = dtstats_df[['Country', 'Districts', 'Communes', 'Cashew_Yield']]
dtstats_df = dtstats_df.sort_values(by=['Districts']).reset_index(drop=True)

# change state names from upper case to title case!!
dtstats_df['Districts'] = dtstats_df['Districts'].str.title()

In [11]:
dist_stats1 = collectionFromImages.reduce(reducer = ee.Reducer.sum()).eq(1).reduceRegions(
  collection = benin_adm1,
  reducer = ee.Reducer.sum(),
  scale = 1000
)
dist_stats1 = dist_stats1.select(['NAME_0','NAME_1', 'sum'], ['Country', 'Districts','Cashew_Yield'], retainGeometry=True).getInfo()

In [12]:
dtstats_df1 = pd.DataFrame()
for dist1 in dist_stats1['features']:
    df1 = pd.DataFrame([dist1['properties']], columns=dist1['properties'].keys())
    dtstats_df1 = pd.concat([dtstats_df1, df1], axis=0)

In [13]:
# Reorder columns in dataframe, sort by state name and reset index
dtstats_df1 = dtstats_df1[['Country', 'Districts', 'Cashew_Yield']]
dtstats_df1 = dtstats_df1.sort_values(by=['Districts']).reset_index(drop=True)

# change state names from upper case to title case!!
dtstats_df1['Districts'] = dtstats_df1['Districts'].str.title()

In [14]:
# ben_map.add_layer(json_layer_ben0)
# ben_map.add_layer(json_layer_ben1)
# ben_map.add_layer(json_layer_ben2)

class Value_changed():
    def __init__(self, dtstats_df):
        self.dtstats_df = dtstats_df
        self.prev = -1
        self.name = ""
        self.bool = True
        self.message3 = HTML()
#         self.marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='white'))
        
#         self.search_control = SearchControl(
#           position="topright",
#           layer=None,
#           zoom=9,
#           property_name='NAME_1',
#           marker=self.marker
#         )
#         ben_map.add_control(self.search_control)
        
    def show_stats(self, b):
        if self.prev==1:
            output_widget2.clear_output()
            y1 = dtstats_df[dtstats_df['Districts']==self.name]['Communes']

            # getting values against each value of y
            plt.figure(figsize=(6,3))
            x1 = dtstats_df[dtstats_df['Districts']==self.name]['Cashew_Yield'] 
            if sum(x1)>1:
                normalized = True
            else:
                normalized = False

            bar_width = 0.5
            opacity = 0.99

            plt.barh(y1, x1, bar_width,
            alpha=opacity,
            color='g',
            label=f'Cashew Tree Cover - {self.name}')



            # setting label of y-axis
            plt.ylabel(f"Communes in {self.name}")

            # setting label of x-axis
            plt.xlabel("Cashew Tree Cover") 
            plt.title(f"Plot of Communes in {self.name} against Cashew Tree Cover")
            plt.legend()
            plt.grid()
            
            fig1, ax1 = plt.subplots(figsize=(6,6))
            colors = ['tab:green', 'tab:cyan', 'tab:gray', 'tab:orange', 'tab:red', 'tab:blue']
    #         explode1 = [0, 0.1, 0.08, 0, 0, 0]
            ax1.pie(x1, labels = y1, colors = colors, autopct='%.0f%%', normalize=normalized)
            ax1.set_title(f'Relative Cashew Tree Cover in {self.name} (Percentage)')
            
            self.message3.value ='''
            <h4>{} Department Cashew Tree Cover Statistics</h4>
            <h4>Total Cashew Tree Cover: {} hectares </h4>
            '''.format(self.name ,round(sum(x1),2))
            with output_widget2:
                display(self.message3)
                plt.show()
                display(button2)
                
        elif self.prev==0:
            output_widget2.clear_output()
            y1 = dtstats_df1[dtstats_df1['Country']=='Benin']['Districts']

            # getting values against each value of y
            plt.figure(figsize=(6,3))
            x1 = dtstats_df1[dtstats_df1['Country']=='Benin']['Cashew_Yield'] 
            if sum(x1)>1:
                normalized = True
            else:
                normalized = False

            bar_width = 0.5
            opacity = 0.99

            plt.barh(y1, x1, bar_width,
            alpha=opacity,
            color='g',
            label=f'Cashew Tree Cover - Benin')



            # setting label of y-axis
            plt.ylabel(f"Departments in Benin")

            # setting label of x-axis
            plt.xlabel("Cashew Tree Cover") 
            plt.title(f"Plot of Departments in Benin against Cashew Tree Cover")
            plt.legend()
            plt.grid()
            
            fig1, ax1 = plt.subplots(figsize=(6,6))
            colors = ['tab:green', 'tab:cyan', 'tab:gray', 'tab:orange', 'tab:red', 'tab:blue']
    #         explode1 = [0, 0.1, 0.08, 0, 0, 0]
            ax1.pie(x1, labels = y1, colors = colors, autopct='%.0f%%', normalize=normalized)
            ax1.set_title(f'Relative Cashew Tree Cover in Benin (Percentage)')
            
            self.message3.value ='''
            <h4>Benin Republic Cashew Tree Cover Statistics</h4>
            <h4>Total Cashew Tree Cover: {} hectares </h4>
            '''.format(round(sum(x1),2))
            with output_widget2:
                display(self.message3)
                plt.show()
                display(button2)
        else:
            output_widget2.clear_output()
            self.message3.value ='''
    <img src="https://recruiting.ultipro.com/TEC1006TESER/JobBoard/18180d88-ced0-4361-bd09-d5eef66dab24/Styles/GetLargeHeaderLogo?brandId=232b9fc7-c10f-4c9a-a06b-56b88ae23f76&m=637395167032660000" width="200" height="40">
    <h4>No Statistics For Benin Communes Yet. View Clip!</h4>
'''
            with output_widget2:
                display(self.message3)
                display(YouTubeVideo('Y2KnXk8iqZU'))
                display(button2)
            
        
    def add_html(self, b):
        html1.value = '''
    <h4>Map Information Terminal</h4>
    <h5>Move pointer to any region in Benin</h5>
'''
        output_widget2.clear_output()
        with output_widget2:
            self.bool = True
            display(html1)
            
    def update_benin(self, feature, **kwargs):
        if self.bool:
            with output_widget2:
                display(button)
            self.bool = False
        html1.value = '''
            <h4>Benin Republic: </h4>
            <h4>Cashew Tree Cover: <b>{} ha</b></h4>
        '''.format(round(sum(dtstats_df1[dtstats_df1['Country']=='Benin'].Cashew_Yield),2))
        
    def update_department(self, feature, **kwargs):
        if self.bool:
            with output_widget2:
                display(button)
            self.bool = False
        self.name = feature['properties']['NAME_1']
        html1.value = '''
            <h4>Admin Type: <b>{}</b></h4>
            <h4>Department Name: <b>{}</b></h4>
            <h4>Cashew Tree Cover: <b>{} ha</b></h4>
        '''.format(feature['properties']['ID_0'], feature['properties']['NAME_1'], round(sum(dtstats_df[dtstats_df['Districts']==self.name].Cashew_Yield),2))
    def update_communes(self, feature, **kwargs):
        if self.bool:
            with output_widget2:
                display(button)
            self.bool = False
        self.name = feature['properties']['NAME_2']
        html1.value = '''
            <h4>Admin Type: <b>{}</b></h4>
            <h4>Department Name: <b>{}</b></h4>
            <h4>Cashew Tree Cover: <b>{} ha</b></h4>
        '''.format(feature['properties']['ID_0'], feature['properties']['NAME_2'], round(sum(dtstats_df[dtstats_df['Communes']==self.name].Cashew_Yield),2))
        
    def value_changed(self, change):
        layer_list = [json_layer_ben0, json_layer_ben1, json_layer_ben2]
        if change.new ==1:
            if self.prev >= 0:
                ben_map.substitute_layer(layer_list[self.prev], layer_list[0])
            else:
                ben_map.add_layer(layer_list[0])
            layer_list[0].on_click(self.update_benin)
            self.prev = 0 
        elif change.new ==2:
            if self.prev >= 0:
                ben_map.substitute_layer(layer_list[self.prev], layer_list[1])
            else:
                ben_map.add_layer(layer_list[1])
            
            layer_list[1].on_click(self.update_department)
            
#             ben_map.remove_control(self.search_control)
# #             self.marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='white'))
#             self.search_control = SearchControl(
#               position="topright",
#               layer=layer_list[1],
#               zoom=9,
#               property_name='NAME_1',
#               marker=self.marker
#             )
#             ben_map.add_control(self.search_control)
            
            self.prev = 1
        elif change.new ==3:
            if self.prev >= 0:
                ben_map.substitute_layer(layer_list[self.prev], layer_list[2])
            else:
                ben_map.add_layer(layer_list[2])
                
            layer_list[2].on_click(self.update_communes)
#             ben_map.remove_control(self.search_control)

# #             self.marker = Marker(icon=AwesomeIcon(name="check", marker_color='green', icon_color='white'))
#             self.search_control = SearchControl(
#               position="topright",
#               layer=layer_list[2],
#               zoom=9,
#               property_name='NAME_2',
#               marker=self.marker
#             )
#             ben_map.add_control(self.search_control)
            self.prev = 2
    
        elif change.new ==0:
            try:
                ben_map.remove_layer(layer_list[self.prev])
            except:
                pass
            self.prev = -1

In [15]:
button = widgets.Button(
    description='Show Statistics',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click here',
    icon='bar-chart' # (FontAwesome names without the `fa-` prefix)
)

In [16]:
button2 = widgets.Button(
    description='Back To Map',
    disabled=False,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click here',
    icon='arrow-left' # (FontAwesome names without the `fa-` prefix)
)

In [17]:
legend_keys = ['Urban', 'Cashew Tree', 'Forest', 'Boundary', 'Nursery']
 #colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
# legend_colors = ['#8DD3C7', '#FFFFB3', '#BEBADA', '#FB8072', '#80B1D3']
legend_colors = [(0, 0, 0), (0, 100, 0), (255, 255, 255), (127, 127, 127), (56, 149, 211)]

ben_map.add_legend(legend_title="Legend", legend_keys=legend_keys, legend_colors=legend_colors, position='topright')

In [18]:
output_widget = widgets.Output(layout={'border': '2px solid green'})
output_control = WidgetControl(widget=output_widget, position='topleft')
ben_map.add_control(output_control)

In [19]:
output_widget2 = widgets.Output(layout={'border': '2px solid green'})
output_control2 = WidgetControl(widget=output_widget2, position='bottomright')
ben_map.add_control(output_control2)

In [20]:
output_widget.clear_output()
logo = widgets.HTML(
    value='<img src="http://www.tnsbenin.org/uploads/1/0/9/8/109816790/logo-cajulab-jpg_orig.jpg" width="150" height="100">'
)
slider1 = widgets.HTML(
    value='''<!DOCTYPE html>
<html>
<head>
<meta name="viewport" content="width=device-width, initial-scale=1">
<style>
.switch {
  position: relative;
  display: inline-block;
  width: 60px;
  height: 34px;
}

.switch input { 
  opacity: 0;
  width: 0;
  height: 0;
}

.slider {
  position: absolute;
  cursor: pointer;
  top: 0;
  left: 0;
  right: 0;
  bottom: 0;
  background-color: #ccc;
  -webkit-transition: .4s;
  transition: .4s;
}

.slider:before {
  position: absolute;
  content: "";
  height: 26px;
  width: 26px;
  left: 4px;
  bottom: 4px;
  background-color: white;
  -webkit-transition: .4s;
  transition: .4s;
}

input:checked + .slider {
  background-color: #006400;
}

input:focus + .slider {
  box-shadow: 0 0 1px #2196F3;
}

input:checked + .slider:before {
  -webkit-transform: translateX(26px);
  -ms-transform: translateX(26px);
  transform: translateX(26px);
}

</style>
</head>
<body>


<img src="https://image.pngaaa.com/459/1311459-middle.png" width="90" height="50">
<label class="switch">
  <input type="checkbox">
  <span class="slider"></span>
</label>
<h4>Cashew Tree Cover</h4><br>

<img src="https://www.clipartmax.com/png/middle/251-2516726_agriculture-natural-environment.png" width="90" height="50">
<label class="switch">
  <input type="checkbox">
  <span class="slider"></span>
</label>
<h4>Nursery Locations</h4><br>

<img src="http://media.natgeotraveller.in/wp-content/uploads/2016/05/Cashew.jpg" width="90" height="50">
<label class="switch">
  <input type="checkbox">
  <span class="slider"></span>
</label>
<h4>Training Information</h4>

</body>
</html> 
'''
)
with output_widget:
    display(logo)
    display(slider1)
    display(benin_bounds)

In [21]:
type_selection = widgets.RadioButtons(
    options=['Cashew Tree Cover', 'Nursery Location', 'Training Area'],
    layout={'width': 'max-content'}, # If the items' names are long
    disabled=False
)
type_select = WidgetControl(widget=type_selection, position='topleft')

message1 = HTML()
message1.value = '''<h4>Nursery1 Info:<br><b>BANTE-Kassin</b></h4> <img src="https://www.clipartmax.com/png/middle/251-2516726_agriculture-natural-environment.png" width="120" height="70">'''

message2 = HTML()
message2.value = '''<h4>Nursery2 Info:<br><b>NIKKI-Assouma</b></h4> <img src="https://www.clipartmax.com/png/middle/251-2516726_agriculture-natural-environment.png" width="120" height="70">'''

message3 = HTML()
message3.value = '''<h4>Nursery3 Info:<br><b>SAVE-Avikpo</b></h4> <img src="https://www.clipartmax.com/png/middle/251-2516726_agriculture-natural-environment.png" width="120" height="70">'''

marker1 = Marker(location=(8.46486667, 1.69925833), rise_on_hover=True, rise_offset = 250, tooltip = "tooltip" , popup=message1)
marker2 = Marker(location=(9.9238767, 3.0556317), rise_on_hover=True, rise_offset = 250, tooltip = "tooltip" , popup=message2)
marker3 = Marker(location=(8.09040333, 2.49072333), rise_on_hover=True, rise_offset = 250, tooltip = "tooltip" , popup=message3)

marker_cluster = MarkerCluster(
    markers=(marker1, marker2, marker3)
)

ben_map.add_layer(marker_cluster)
ben_map.addLayer(collectionFromImages, {'min':0, 'max': 4, 'palette': "black, green, white, gray"}, 'collection map')


# def layer_select(change):
#     if change.new == 'Cashew Tree Cover':
        
#         try:
#             ben_map.remove_layer(marker_cluster)
#         except:
#             pass
        
#         try:
#             with output_widget:
#                 display(benin_bounds)
#             ben_map.addLayer(collectionFromImages, {'min':0, 'max': 4, 'palette': "black, green, white, gray"}, 'collection map')
            
#         except:
#             pass
        
#     elif change.new == 'Nursery Location':
        
#         try:
#             ben_map.remove_layer('collection map')
#             ben_map.remove_control(benin_bounds)
#         except:
#             pass
        
#         try:
#             ben_map.add_layer(marker_cluster)
#         except:
#             pass
#     elif change.new == 'Training Area':
#         pass
        

In [22]:
#ben_map.add_control(WidgetControl(widget=benin_bounds, position='topleft'))
val = Value_changed(dtstats_df)
benin_bounds.observe(val.value_changed, 'value')
# type_selection.observe(layer_select, 'value')

In [23]:
# output_widget.append_display_data(YouTubeVideo('Y2KnXk8iqZU'))

In [24]:
# coll_BEN = collectionFromImages.reduce(reducer = ee.Reducer.sum()).clipToCollection(benin_adm0)

# # colline_BEN = colline_BEN.clipToCollection(nigeria)

# # Define arbitrary thresholds on the 6-bit nightlights image.
# zones = coll_BEN.eq(1)
# zones = zones.updateMask(zones.neq(0))

# # Convert the zones of the thresholded nightlights to vectors.
# vectors = zones.addBands(coll_BEN).reduceToVectors(
#   geometry = benin_adm0,
#   #crs = coll_BEN.projection(),
#   scale = 1000,
#   eightConnected = True,
#   reducer = ee.Reducer.mean()
# )

# # Display the thresholds.
# ben_map.setCenter(2.4, 9.0, 9)
# ben_map.addLayer(zones, {'palette': 'darkred', 'bias':3000}, 'raster')
# ben_map.addLayer(coll_BEN, {'min':0, 'max': 4, 'palette': "black, green, white, gray"}, 'collection map')

# # ben_map.add_layer(marker_cluster)

# # # # Make a display image for the vectors, add it to the map.
# # display = ee.Image(0).updateMask(0).paint(vectors, '000000', 1)
# # ben_map.addLayer(display, {'palette': '000000'}, 'vectors')

In [25]:
output_widget2.clear_output()
with output_widget2:
    display(html1)

In [26]:
button.on_click(val.show_stats)

In [27]:
button2.on_click(val.add_html)

In [28]:
ben_map

Map(center=[9.0, 2.4], controls=(FullScreenControl(options=['position'], position='topright'), ZoomControl(opt…